In [1]:
import skimage
from skimage import data, io, filters
from skimage.color import rgb2gray, gray2rgb

import matplotlib.pyplot as plt

from skimage import data
from skimage.color import rgb2hed
import skimage
import skimage.morphology
import skimage.exposure
import skimage.util
import imageio
from matplotlib.colors import LinearSegmentedColormap
import matplotlib
import numpy as np

from skimage.filters import threshold_otsu

from skimage.morphology import diamond, binary_dilation, closing, square

from skimage.segmentation import clear_border
from skimage.measure import label, regionprops

import math

In [2]:
#The crop is not the part of the algorithm
im_ce = io.imread("CK-DAB-H-HR2-16PgR-B.png")[6000:12000, ...]
im_he = io.imread("HE-HR2-16PgR-B.png")[6000:12000, ...]

/usr/local/lib/python3.5/dist-packages/PIL/Image.py:2735: DecompressionBombWarning: Image size (91115640 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


In [3]:
def rgb2h(im):
    im_rgb = im[:,:,:3]
    im_hed = rgb2hed(im_rgb)
    im_h = im_hed[:, :, 0]
    return skimage.exposure.rescale_intensity(im_h, out_range=(0,1))

In [4]:
im_ce_gray = rgb2gray(im_ce)
im_he_gray = rgb2gray(im_he)

im_ce_h = rgb2h(im_ce)
im_he_h = rgb2h(im_he)

### Segmentation

In [5]:
#Returns a part of an image given coordinates

def get_subimage(coords, im, relative_coords=(0,0)):
    return im[coords[0]+relative_coords[0]:coords[2]+relative_coords[1], 
              coords[1]+relative_coords[0]:coords[3]+relative_coords[1]]

The segmentation is based on finding objects of a minimal size. Objects are represented by the class Segment.

In [6]:
class Segment:
    def __init__(self, region):
        self.region = region   

    def get_subimage(self, im):
        return im[self.region.bbox[0]:self.region.bbox[2], 
                  self.region.bbox[1]:self.region.bbox[3]]
    
    def get_min_coords(self):
        return self.region.bbox[0], self.region.bbox[1]
    
    def get_max_coords(self):
        return self.region.bbox[2], self.region.bbox[3]
    
    def get_area(self):
        return self.region.area

In [7]:
#Returns a list of objects larger than min_area

def segment_binary_objects(im, min_area):
    bw = closing(im, square(3))
    cleared = clear_border(bw)
    label_image = label(cleared)

    return [Segment(region) for region in regionprops(label_image) if (region.area >= min_area)]

In [8]:
#Applies the thresholding (Otsu) twice to remove too small and too large values

def double_otsu_threshold(im):
    thr1 = threshold_otsu(im)
    im_thr1 = im * (im < thr1)
    thr2 = threshold_otsu(im_thr1)
    return (im_thr1 > thr2)

In [9]:
(1,2)+np.ones((2,3)).shape[2:]

(1, 2)

In [10]:
#Adds an empty margin to an image.

def add_margin(im, mw):
    x = np.zeros((im.shape[0]+(2*mw), im.shape[1]+(2*mw))+im.shape[2:])
    x[mw:im.shape[0]+mw, mw:im.shape[1]+mw] += im
    return x

In [11]:
#The number of columns of the samples
NUMBER_OF_COLUMNS = 3
#Minimum object size in the input image
MIN_OBJECT_SIZE_ORIGINAL = 100
#Implementation detail: To remove borders around a sample, we identify the very large object in the center
MIN_OBJECT_SIZE_SECOND = 2000
#The object searching algorithm (regionprops) does not like objects touching the border of the image, so we add a small margin
CORRECTION_MARGIN = 10

In [12]:
def get_segments(im_h, im_gray):
    segments_unsorted = segment_binary_objects(add_margin(im_gray > 0, CORRECTION_MARGIN), 100)
    
    #Sorts the samples (by rows)
    segments_sorted = sum([
        sorted(
            sorted(segments_unsorted, key=lambda x:x.get_min_coords()[0])[i:i+NUMBER_OF_COLUMNS], 
            key=lambda x: x.get_min_coords()[1]) 
        for i in range(0, math.floor(len(segments_unsorted)), NUMBER_OF_COLUMNS)],[])
    
    im_segments = list(map(lambda s: double_otsu_threshold(s.get_subimage(add_margin(im_h, CORRECTION_MARGIN))), segments_sorted))

    #This part removes the margins in the already segmented samples
    segments2 = list(map(lambda i: segment_binary_objects(add_margin(binary_dilation(i, diamond(10)), CORRECTION_MARGIN), 2000)[0], 
                       im_segments))

    im_segments2 = list(map(lambda si: si[0].get_subimage(add_margin(si[1], CORRECTION_MARGIN)), zip(segments2, im_segments)))

    #Get coordinates of the samples in the original image
    segment_coordinates = list(map(lambda x: (x[0].get_min_coords()[0] - 2*CORRECTION_MARGIN + x[1].get_min_coords()[0],
                                         x[0].get_min_coords()[1] - 2*CORRECTION_MARGIN + x[1].get_min_coords()[1],
                                         x[0].get_min_coords()[0] - 2*CORRECTION_MARGIN + x[1].get_max_coords()[0],
                                         x[0].get_min_coords()[1] - 2*CORRECTION_MARGIN + x[1].get_max_coords()[1]), 
                              zip(segments_sorted, segments2))) 
    
    return list(zip(im_segments2, segment_coordinates))

In [13]:
segments_ce = get_segments(im_ce_h, im_ce_gray)
segments_he = get_segments(im_he_h, im_he_gray)

### Registration using SimpleElastix

In [159]:
IMAGE_INDEX = 0

In [161]:
import SimpleITK as sitk

In [162]:
im_he_h_ = get_subimage(segments_he[IMAGE_INDEX][1], im_he_h) * segments_he[IMAGE_INDEX][0]
im_ce_h_ = get_subimage(segments_ce[IMAGE_INDEX][1], im_ce_h) * segments_ce[IMAGE_INDEX][0]

In [163]:
elastixImageFilter = sitk.ElastixImageFilter()

In [164]:
im_fixed = sitk.GetImageFromArray(im_he_h_.astype(np.float32))
elastixImageFilter.SetFixedImage(im_fixed);

im_moving = sitk.GetImageFromArray(im_ce_h_.astype(np.float32))
elastixImageFilter.SetMovingImage(im_moving);

In [165]:
elastixImageFilter.SetParameterMap(sitk.GetDefaultParameterMap("translation"))
elastixImageFilter.AddParameterMap(sitk.GetDefaultParameterMap('affine'))
elastixImageFilter.Execute()

<SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x7eff7186ded0> >

In [166]:
im_result = elastixImageFilter.GetResultImage()

In [167]:
im_fixed_np = sitk.GetArrayFromImage(im_fixed)
im_moving_np = sitk.GetArrayFromImage(im_moving)
im_result_np = sitk.GetArrayFromImage(im_result)

In [168]:
io.imsave("test.tif", im_result_np)

### Registration (naive, just translation + squared error)

The registration algorithm is extremely naive. Just add a margin to the he image, and move the ce image around minimizing the squared loss.

In [ ]:
def get_loss(im1, im2, x, y):
    xx, yy = im2.shape
    cut = im1[x:(x+xx), y:(y+yy)]
    loss = np.sum((cut - im2)**2)
    print(x, y, loss)
    return x, y, loss

#This gives the relative coordinates of im2 (supposed to be ce) in im1

def get_relative_coords(im1, im2):
    print("Image shapes: ", im1.shape, im2.shape)
    l = [get_loss(im1, im2, x, y) for x in range(0, im1.shape[0]-im2.shape[0]) for y in range(0, im1.shape[1]-im2.shape[1])]
    b = sorted(l, key=lambda x: x[2])[0]
    return (b[0], b[1])

def max_diff(im1, im2):
    x, y = im1.shape
    u, v = im2.shape
    return max(max(u-x, v-y), 0)

SEARCH_MARGIN = 10

#Sizes of the margins added to he samples to be able to move ce samples around
margin_he = list(map(lambda x: max_diff(x[0][0], x[1][0])+SEARCH_MARGIN, zip(segments_he, segments_ce)))

#The registration: Gives the relative coordinates of ce in the he with the added margin

relative_coords_ce_in_he = list(map(lambda he, ce, marg: np.array(get_relative_coords(add_margin(he[0], marg), ce[0])), 
                               segments_he, segments_ce, margin_he))

# Obtaining registered images

#Just choose which sample you want to use
IMAGE_INDEX = 10

def add_inside_image(im, coords, im_to_add):
    z1 = im_to_add.shape[0]
    z2 = im_to_add.shape[1]
    c1 = coords[0]
    c2 = coords[1]

    im[c1:c1+z1, c2:c2+z2] += im_to_add
    
    return im

### superimpose gray CE and gray HE

In [ ]:
result_he = add_margin(get_subimage(segments_he[IMAGE_INDEX][1], im_he_gray), margin_he[IMAGE_INDEX])

In [ ]:
result_ce = add_inside_image(np.zeros(result_he.shape), 
                             relative_coords_ce_in_he[IMAGE_INDEX], 
                             get_subimage(segments_ce[IMAGE_INDEX][1], im_ce_gray))

In [ ]:
im_superimposed = ([0,1,0] * gray2rgb(np.array(result_he, dtype=float)) * 0.5 + 
          [1,0,0] * gray2rgb(np.array(result_ce, dtype=float)) * 0.5)

In [ ]:
io.imsave("test.png", im_superimposed)

### Fill holes in CE

In [172]:
def rgb2dab(im):
    im_rgb = im[:,:,:3]
    im_hed = rgb2hed(im_rgb)
    im_h = im_hed[:, :, 2]
    return skimage.exposure.rescale_intensity(im_h, out_range=(0,1))

In [173]:
i_ce_dab_mask = double_otsu_threshold(rgb2dab(get_subimage(segments_ce[IMAGE_INDEX][1], im_ce)))
i_ce_h_mask = double_otsu_threshold(rgb2h(get_subimage(segments_ce[IMAGE_INDEX][1], im_ce)))

In [174]:
labels = skimage.measure.label(1-i_ce_dab_mask.astype(int))
regs = skimage.measure.regionprops(labels)

In [175]:
MIN_AREA=100
MAX_AREA=200
H_PROPORTION=0.4

In [176]:
holes=np.zeros(i_ce_dab_mask.shape)
for region in regs:
    if region.area<MAX_AREA:
        coords = tuple(zip(*region.coords))
        if region.area<MIN_AREA:
            holes[coords]=1
        else:
            H_values = (1-i_ce_h_mask)[coords]
            meanH = np.mean(H_values)
            if meanH >= H_PROPORTION:
                holes[coords]=1        

filled_holes = holes + i_ce_dab_mask

In [177]:
filled_holes

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [178]:
filled_holes.shape

(848, 748)

### superimpose HE and CE with filled holes (elastix registration)

In [179]:
filled_holes_elastix_im = sitk.GetImageFromArray(filled_holes.astype(np.float32))

In [180]:
result = sitk.Transformix(filled_holes_elastix_im, elastixImageFilter.GetTransformParameterMap())
filled_holes_result_im = sitk.GetArrayFromImage(result)

In [ ]:
io.imsave("test.tif", im_composed)

In [181]:
he = get_subimage(segments_he[IMAGE_INDEX][1], im_he)/255

In [182]:
he_masked_with_ce_with_filled_holes = np.stack([he[:,:,0],
         he[:,:,1]+ filled_holes_result_im,
         he[:,:,2]],
        axis=2)

In [183]:
io.imsave("he_masked_with_ce_with_filled_holes_elastix_registration.png", he_masked_with_ce_with_filled_holes)

Lossy conversion from float64 to uint8. Range [-0.2503100633621216, 2.375198420356302]. Convert image to uint8 prior to saving to suppress this warning.


### superimpose HE and CE with filled holes (naive registration)

In [ ]:
he = add_margin(get_subimage(segments_he[IMAGE_INDEX][1], im_he), margin_he[IMAGE_INDEX])/255

In [ ]:
filled_holes_ext = add_inside_image(np.zeros(he.shape[:2]), 
                             relative_coords_ce_in_he[IMAGE_INDEX], 
                             filled_holes)

In [ ]:
io.imshow(he)

In [ ]:
io.imshow(filled_holes_ext)

In [ ]:
he_masked_with_ce_with_filled_holes = np.stack([0.8*he[:,:,0],
         0.2*he[:,:,1]+ 0.8*filled_holes_ext,
         0.8*he[:,:,2]],
        axis=2)

In [ ]:
io.imshow(he_masked_with_ce_with_filled_holes)

In [ ]:
io.imsave("he_masked_with_ce_with_filled_holes_naive_registration.png", he_masked_with_ce_with_filled_holes)